# Importation des packages 

Nous chargons les modules necessaires pour notre analyse. 

In [1]:
#Importation des packages

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes
import os 

In [2]:
# Definir le repertoire de travail 
pwd = "C:\\Users\\mouss\\OneDrive\\Documents\\TotalEnergies" 
os.chdir(pwd) 
 
# Afficher le nouveau repertoire de travail 
print(os.getcwd()) 

C:\Users\mouss\OneDrive\Documents\TotalEnergies


# Données
Nous chargeons le jeu de données des produits. Cette table contient 8 472 lignes et 6 5. Tous les facteurs sont des variables à modalités. 

In [3]:
# Importation des données

data = pd.read_csv("mapping_product.csv",sep = ";"
                       #,index_col = "EP_CODE"
                      , encoding= 'latin-1')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8472 entries, 0 to 8471
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PRODUCT_CODE               8472 non-null   object
 1   PRODUCT_GRAND_FAMILY_CODE  8294 non-null   object
 2   PRODUCT_GRAND_FAMILY_NAME  8294 non-null   object
 3   PRODUCT_FAMILY_CODE        8294 non-null   object
 4   PRODUCT_FAMILY_NAME        8294 non-null   object
dtypes: object(5)
memory usage: 331.1+ KB


# Echantillon
Ci dessous un échantilon de cette table.

In [5]:
data.head()

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_GRAND_FAMILY_NAME,PRODUCT_FAMILY_CODE,PRODUCT_FAMILY_NAME
0,732,GF05,Frais de services,FA03,Abonnements
1,733,GF04,Produits Hors Station,FA05,Tr Autres tiers - Ristournes et commissions
2,734,GF04,Produits Hors Station,FA05,Tr Autres tiers - Ristournes et commissions
3,735,GF07,Caution et Garantie,FA15,CA Tr Autres tiers - Caution/Garantie
4,738,GF04,Produits Hors Station,FA17,Tr Parkings


# Nettoyage et preparation de données

## Types des variables

Dans cette partie, il s'agit de regarder de plus près le type de chaque variable et decider s'il est pertinent de les convertir vers d'autres types. Pour ce faire, analysons la sortie suivante : 

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8472 entries, 0 to 8471
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PRODUCT_CODE               8472 non-null   object
 1   PRODUCT_GRAND_FAMILY_CODE  8294 non-null   object
 2   PRODUCT_GRAND_FAMILY_NAME  8294 non-null   object
 3   PRODUCT_FAMILY_CODE        8294 non-null   object
 4   PRODUCT_FAMILY_NAME        8294 non-null   object
dtypes: object(5)
memory usage: 331.1+ KB


On remaruqe que toute les variables sont de type *object* (correspondant à string). Le type *object* ne pose aucun problème pour les variables qui sont de nature categorielles. C'est le cas pour toutes les variables dans cette table.

## Nombre de modalités par variables

In [15]:
# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

unique_counts = pd.DataFrame.from_records([(col, data[col].nunique()) for col in data.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_uniques']).sort_values(by=['Nombre_valeurs_uniques'])

unique_counts

,Nom_variables,Nombre_valeurs_uniques
1,PRODUCT_GRAND_FAMILY_CODE,8
2,PRODUCT_GRAND_FAMILY_NAME,8
3,PRODUCT_FAMILY_CODE,25
4,PRODUCT_FAMILY_NAME,25
0,PRODUCT_CODE,8472


## Données manquantes

Dans cette table, il y'a peu de données manquantes. Toutes les variables hormis PRODUCT_CODE ont des données manquantes et le même nombre : 178 soit 2.1%. Il s'agit très certainement des mêmes individus pour toutes les variables. Donc en omettant ces données manquantes, on supprimera 178 lignes de notre jeu de données. 

In [21]:
# Données manquantes

# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

NA_counts = pd.DataFrame.from_records([(col
                                        , data[col].isna().sum()
                                       , round((data[col].isna().sum())/(data[col].size),3)) for col in data.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_manquantes',"%NA"]).sort_values(by=['Nombre_valeurs_manquantes'], ascending=False)

NA_counts

,Nom_variables,Nombre_valeurs_manquantes,%NA
0,PRODUCT_CODE,0,0.0
1,PRODUCT_GRAND_FAMILY_CODE,0,0.0
2,PRODUCT_GRAND_FAMILY_NAME,0,0.0
3,PRODUCT_FAMILY_CODE,0,0.0
4,PRODUCT_FAMILY_NAME,0,0.0


In [22]:
data = data.copy().dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8294 entries, 0 to 8471
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PRODUCT_CODE               8294 non-null   object
 1   PRODUCT_GRAND_FAMILY_CODE  8294 non-null   object
 2   PRODUCT_GRAND_FAMILY_NAME  8294 non-null   object
 3   PRODUCT_FAMILY_CODE        8294 non-null   object
 4   PRODUCT_FAMILY_NAME        8294 non-null   object
dtypes: object(5)
memory usage: 388.8+ KB


# Description sommaire des données

Le tableau ci dessous decrit brievement chaque variable. Il y'a dans ce tableau pour chque variable :
* count : le nombre d'observations (exclu les données manquantes :NAN)
* unique : le nombre de valeurs (modalités) discincts
* top : la valeur la plus fréquente
* freq : la frequence de la valeur la plus frequente (nombre brute=> comptage)
* freq_p : la proportion de la valeur la plus frequente ( en %)

Ainsi, on peut remarquer que les produits de type **Abonnements** sont majoritairement vendus (>90%). 

Les recettes proviennes donc des **frais de services**.


In [15]:
stats = data.describe()
stats

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_GRAND_FAMILY_NAME,PRODUCT_FAMILY_CODE,PRODUCT_FAMILY_NAME
count,8472,8294,8294,8294,8294
unique,8472,8,8,25,25
top,X6IA,GF05,Frais de services,FA03,Abonnements
freq,1,7816,7816,7657,7657


In [16]:
# Ajout de la frequence en %

stats.loc["freq_p"] = pd.Series(stats.loc["freq"]/stats.loc["count"]).apply(lambda value : round(value,3)) # frequence en pourcentage
#stats = pd.DataFrame(np.insert(stats.values, stats.shape[0], freq_p, axis=0))
stats

,PRODUCT_CODE,PRODUCT_GRAND_FAMILY_CODE,PRODUCT_GRAND_FAMILY_NAME,PRODUCT_FAMILY_CODE,PRODUCT_FAMILY_NAME
count,8472,8294,8294,8294,8294
unique,8472,8,8,25,25
top,X6IA,GF05,Frais de services,FA03,Abonnements
freq,1,7816,7816,7657,7657
freq_p,0.0,0.942,0.942,0.923,0.923


# Autres analyses

On remarque dans le tableau ci-dessous que les produits les plus vendus sont de la famille **Abonnements** avec plus de **92%** des transactions.

In [23]:
data.value_counts("PRODUCT_FAMILY_NAME", normalize=True)

PRODUCT_FAMILY_NAME
Abonnements                                     0.923197
Tr Péages                                       0.014468
Frais à l acte                                  0.013504
Transactions stations carburants                0.010490
Transactions Station Boutique                   0.007475
Transactions Station Lavage                     0.004099
Famille Produits Non Utilisés                   0.003738
Frais sur transactions Péages                   0.003617
Tr Autres tiers - Services Assistance 24/24     0.003617
Tr Autres tiers - CA C.H.R.                     0.003497
Remboursement divers                            0.002170
Tr Parkings                                     0.002050
Ristournes et commissions carburants            0.001929
CA Tr Autres tiers - Caution/Garantie           0.001447
Frais sur transactions Stations                 0.001206
Tr Electriques                                  0.001085
Tr Autres tiers - Ristournes et commissions     0.000603
Ecotaxe    

# Enregistrer les données propres

In [20]:
data.to_csv('./data/mapping_product.csv',sep=';',encoding='latin-1',index=False)